In [ ]:
#| hide

# Project Overview

> Likelihood To Buy A Pass (LTBP)

See example sections for how to be able to get your project to a state that will mirror this one

## Install

```sh
pip install LTBP
```

## LTBP Use Cases

In [ ]:
#| hide

import logging, sys
logging.disable(sys.maxsize)

### Where results of LTR lives:

**Current Fiscal Season Results:**
Development Results: MachineLearningOutPuts.DEV.LTBP_PREDICTIONS

> Note: Development results will be something that we move around as more scale begins to happen, but that isn't solved as of now.

Development Results: MachineLearningOutPuts.LTBP.LTBP_PREDICTIONS

In [ ]:
from LTBP.data.utils import snowflake_query

sf = snowflake_query()

display(sf.run_sql_str("SELECT COUNT(1) AS TOTAL_POP FROM MachineLearningOutPuts.LTBP.LTBP_PREDICTIONS"))

display(sf.run_sql_str("SELECT TOP 10 * FROM MachineLearningOutPuts.LTBP.LTBP_PREDICTIONS"))

,TOTAL_POP
0,14759852


/Users/jeremydemlow/miniforge3/envs/ltbp/lib/python3.9/site-packages/data_system_utilities/snowflake/query.py:73: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,ECID,PROBABILITY
0,61068721,0.038751
1,130053505,0.492399
2,49940527,0.051959
3,74112965,0.034402
4,32759861,0.004805
5,57941408,0.002742
6,13674145,0.065647
7,60909761,0.009027
8,97172783,0.002126
9,131977740,0.109056


The output of LTBP_PREDICTIONS Table

**Schema:**

|Variable Name| Description|
|:---|:---|
|ECID | Customer Key|
|PROBABILITY| The estimated probability that this ECID will buy a Pass in this Fiscal Year|

**Label Logic:**

LTBP is composed of five bins low, low-med, medium, med-high high. In the past, there was an attempt to optimize the return rate to be heavy in low, but for simplicity and scalability of this project, DSDE decided to use NTILE, which creates equal buckets for the five labeled groups.

In [ ]:
display(sf.run_sql_str("""
SELECT 
LikelihoodToBuyPassLabel,
COUNT(1) as Total
FROM (
SELECT ECID
, PROBABILITY
, ntile(5) over (ORDER BY PROBABILITY) as ntile_5
, CASE WHEN ntile_5 = 5 THEN 'High'
  WHEN ntile_5 = 4 THEN 'Med-High'
  WHEN ntile_5 = 3 THEN 'Medium'
  WHEN ntile_5 = 2 THEN 'Low-Med'
  WHEN ntile_5 = 1 THEN 'Low'
  END AS LikelihoodToBuyPassLabel
FROM MACHINELEARNINGOUTPUTS.ltbp.ltbp_PREDICTIONS ltbp
ORDER BY PROBABILITY DESC
)
GROUP BY LikelihoodToBuyPassLabel
"""))

/Users/jeremydemlow/miniforge3/envs/ltbp/lib/python3.9/site-packages/data_system_utilities/snowflake/query.py:73: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,LIKELIHOODTOBUYPASSLABEL,TOTAL
0,High,2951970
1,Medium,2951970
2,Med-High,2951970
3,Low-Med,2951971
4,Low,2951971


In [ ]:
display(sf.run_sql_str("""SELECT ECID
, PROBABILITY
, ntile(5) over (ORDER BY PROBABILITY) as ntile_5
, CASE WHEN ntile_5 = 5 THEN 'High'
  WHEN ntile_5 = 4 THEN 'Med-High'
  WHEN ntile_5 = 3 THEN 'Medium'
  WHEN ntile_5 = 2 THEN 'Low-Med'
  WHEN ntile_5 = 1 THEN 'Low'
  END AS LikelihoodToBuyPassLabel
FROM MACHINELEARNINGOUTPUTS.ltbp.ltbp_PREDICTIONS ltbp
ORDER BY PROBABILITY DESC
LIMIT 10
"""))

/Users/jeremydemlow/miniforge3/envs/ltbp/lib/python3.9/site-packages/data_system_utilities/snowflake/query.py:73: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,ECID,PROBABILITY,NTILE_5,LIKELIHOODTOBUYPASSLABEL
0,131035762,0.999566,5,High
1,112628819,0.999552,5,High
2,138473097,0.999544,5,High
3,102515974,0.999541,5,High
4,136960602,0.999527,5,High
5,131977201,0.999525,5,High
6,137118754,0.999525,5,High
7,137071085,0.999522,5,High
8,131795763,0.999518,5,High
9,58857511,0.999517,5,High
